In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import random
from utils.read_matches import *
from utils.misc import *
import pyimplicitdist
import poselib # https://github.com/vlarsson/PoseLib
import matplotlib.pyplot as plt

In [ ]:
# Read queries
queries = read_matches_list('data/')

images = [
    plt.imread('data/' + q['name'])
    for q in queries
]


# Examples
The data folder contains 5 example 2D-3D queries from a fisheye camera.  You can change `query_idx` below to view different examples (0 to 4).

In [ ]:
query_idx = 4
query = queries[query_idx]
im = images[query_idx]

im_height, im_width, _ = im.shape

# Initialize principal point to center
initial_pp = [im_width / 2, im_height / 2]    

# Get 2D-3D matches and center keypoints
p2d, p3d = query['matches']

fig = plt.figure(figsize=(8, 8))
fig.add_subplot(2,1,1)
plt.imshow(im)
plt.plot(p2d[:,0], p2d[:,1], 'r.', markersize=1)
ax = fig.add_subplot(2,1,2,projection='3d')
ax.plot3D(p3d[:,0], p3d[:,1], p3d[:,2],'.', markersize=1)

# 1D Radial camera estimation

First we estimate a 1D radial camera, giving us the orientation and first two components of the translation vector. Below we use poselib to do the estimation, followed by non-linear refinement of the principal point

In [ ]:
threshold = 6.0 # px
            
# Initial estimation of radial camera using poselib
p2d_center = [x - initial_pp for x in p2d]
poselib_pose, info = poselib.estimate_1D_radial_absolute_pose(p2d_center, p3d, {"max_reproj_error":  threshold})

# Get inlier correspondences (to the 1D radial camera)
p2d_inlier = p2d[info["inliers"]]
p3d_inlier = p3d[info["inliers"]]


# Refine principal point
initial_pose = pyimplicitdist.CameraPose()
initial_pose.q_vec = poselib_pose.q
initial_pose.t = poselib_pose.t
out = pyimplicitdist.pose_refinement_1D_radial(p2d_inlier, p3d_inlier, initial_pose, initial_pp, pyimplicitdist.PoseRefinement1DRadialOptions())
refined_initial_pose, pp = out['pose'], out['pp']


## Visualizing the radial reprojection errors
Below we visualize the radial reprojection before, both before and after principal point refinement. Refining the principal point improves the estimate slightly.

In [ ]:
fig = plt.figure(figsize=(8, 8))
ind = random.sample(range(len(p2d_inlier)), 20)
plt.imshow(im)
plt.plot(p2d_inlier[ind,0], p2d_inlier[ind,1], 'bo')
plot_1d_radial_reprojs(qvec2rotmat(initial_pose.q_vec), initial_pose.t, initial_pp, p3d_inlier[ind], im_width, im_height,color='r')
plot_1d_radial_reprojs(qvec2rotmat(refined_initial_pose.q_vec), refined_initial_pose.t, pp, p3d_inlier[ind], im_width, im_height,color='g')
plt.legend(['2D points', 'Initial 1D est.', 'Refined 1D est.'])

# TODO Fix legend


# Estimating the forward translations
To perform the pose-refinement with the implicit distortion model, we first build the `CostMatrix` struct. This contains the neighbourhood information and interpolation coefficients necessary for the regularizer. 

Calling `pyimplicitdist.pose_refinement` optimize the full 6-DoF (including the forward translation!).

In [ ]:
cm_opt = pyimplicitdist.CostMatrixOptions()
refinement_opt = pyimplicitdist.PoseRefinementOptions()

cost_matrix = pyimplicitdist.build_cost_matrix(p2d_inlier, cm_opt, pp)
pose = pyimplicitdist.pose_refinement(p2d_inlier, p3d_inlier, cost_matrix, pp, refined_initial_pose, refinement_opt)

# Some simple median-based filtering to remove outliers
out = pyimplicitdist.filter_result_pose_refinement(p2d_inlier, p3d_inlier, pose, pp, refinement_opt)
p2d_filter, p3d_filter = np.array(out["points2D"]), np.array(out["points3D"])



## Visualizing the estimated pose

The estimated (green) and ground-truth (red) camera pose are visualized below. Since this data is fairly easy, they overlap almost exactly.

In [ ]:
gt_q = query['qvec']
gt_t = query['tvec']


# For visualization we restrict ourselves to point close to the GT camera
gt_cc = -qvec2rotmat(gt_q).T @ gt_t
dist = [np.linalg.norm(X - gt_cc) for X in p3d_filter]
ind = dist < 3*np.median(dist)

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection='3d')
#plot_camera(ax, pose.q_vec, pose.t, 10)
ax.plot3D(p3d_filter[ind,0],p3d_filter[ind,1],p3d_filter[ind,2],'b.',markersize=1)
plot_camera(ax, gt_q, gt_t, 1.5, 'r')
plot_camera(ax, pose.q_vec, pose.t, 2, 'g')




# Recovering an explicit intrinsic calibration
Given a camera pose and 2D-3D correspondences we can estimate an explicit (non-parametric) calibration.

Here we tune the regularization strength automatically to balance the radial and tangential components of the error.

In [ ]:
# Estimate an explicit intrinsic calibration using the estimated pose

# Re-build cost-matrix for the filtered points
cost_matrix = pyimplicitdist.build_cost_matrix(p2d_filter, cm_opt, pp)
calib = pyimplicitdist.calibrate(p2d_filter, p3d_filter, cost_matrix, pp, pose)

print(calib)

## Visualizing the intrinsic calibration
The plot below shows the estimated intrinsic calibration mapping (red) against the ground-truth parametric model.

In [ ]:
r_f = np.array(calib.r_f)
theta_r = np.array(calib.theta_r)

theta_min = np.min(theta_r[:,0])
theta_max = np.max(theta_r[:,0])

# Compute mapping for the GT parametric model
dist_params = query['camera']['params']
pp_gt = query['camera']['params'][2:4]
thetas = np.linspace(theta_min, theta_max,20)
r_gt = np.array([
        np.linalg.norm(x-pp_gt)
        for x in apply_opencv_distortion(
            [np.array([np.sin(theta),0,np.cos(theta)]) for theta in thetas],
        dist_params)
        ])
f_gt = r_gt / np.tan(thetas)

# Compute the "raw" mapping
R = qvec2rotmat(pose.q_vec)
Xcam = np.array([R @ Z + pose.t for Z in p3d_filter])

r_raw = [np.linalg.norm(x - pp) for x in p2d_filter]
f_raw = [r*r * Z[2] / np.dot(x-pp, Z[0:2]) for (r,x,Z) in zip(r_raw, p2d_filter, Xcam)]

fig = plt.figure(figsize=(8, 8))
plt.subplot(1,2,1)
plt.plot(r_gt,f_gt,'bo')
plt.plot(r_f[:,0], r_f[:,1],'r-')
plt.legend(['GT calib','Est. calib'])
plt.xlabel('image radius r_i')
plt.ylabel('pointwise focal length f_i')

plt.subplot(1,2,2)
plt.plot(r_raw,f_raw,'.',markersize=1)
plt.legend(['Raw f_i'])
plt.xlabel('image radius r_i')
plt.ylabel('pointwise focal length f_i')


## Reprojecting into the image
Calling the function `pyimplicitdist.distort(points3D, calib)` we can project into the image. Note that the 3D points should be in the camera coordinate system (i.e. after applying R,t).

In [ ]:
# Project into the image with the estimated calibration

R = qvec2rotmat(pose.q_vec)
Xcam = np.array([R @ Z + pose.t for Z in p3d_filter])

proj = np.array(pyimplicitdist.distort(Xcam, calib))

fig = plt.figure(figsize=(8, 8))
plt.imshow(im)
plt.plot(proj[:,0], proj[:,1],'r.')
plt.plot(p2d_filter[:,0], p2d_filter[:,1],'g+')


We can also use the estimated mapping to undistort the image.

In [ ]:
target_fov = 120 / 180 * np.pi
subsample = 5
target_h = int(np.ceil(im_height / subsample))
target_w = int(np.ceil(im_width / subsample))

target_focal = target_w / 2 / np.tan(target_fov/2)

ii, jj = np.meshgrid(np.linspace(1,target_w,target_w), np.linspace(1,target_h,target_h), indexing='ij')
ii = ii - 0.5
jj = jj - 0.5

# Remove principal point
ii = ii - target_w / 2
jj = jj - target_h / 2


X = np.c_[ ii.flatten(), jj.flatten(), target_focal * np.ones(np.prod(ii.shape)) ]
x = np.array(pyimplicitdist.distort(X, calib))
im_undist = np.zeros((target_h, target_w, 3), dtype=im.dtype)
for i in range(target_w):
    for j in range(target_h):
        coord = x[j + i*target_h]
        if np.any(coord < 0) or coord[1] > im_height-1 or coord[0] > im_width-1:
            continue          
        if np.any(np.isnan(coord)):
            continue
        im_undist[j,i] = im[int(coord[1]), int(coord[0])]
        

fig = plt.figure(figsize=(8, 8))
plt.subplot(2,1,1)
plt.imshow(im_undist)
plt.subplot(2,1,2)
plt.imshow(im)


